In [1]:
# Import all the necessary packages
import numpy as np
import nibabel as nib                                                     # Read / write access to some common neuroimaging file formats
import itk                                                                # Jupyter widgets to visualize images in 2D and 3D
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
import cv2

%matplotlib inline
# import seaborn as sns
# sns.set_style('darkgrid')

In [2]:
def apply_window(img, width=500, center=40):
    # np.interp(a, (a.min(), a.max()), (-1, +1))

    # convert below window to black
    img[img<(center-(width/2))]=center-(width/2)
    # convert above window to white
    img[img>(center+(width/2))]=center+(width/2)

    # normalize image
    img_scaled = np.interp(img, (img.min(), img.max()), (0, +1))
    # print(img_scaled.shape)
    # print(np.min(img_scaled), np.max(img_scaled))
    return img

## Original Images

In [3]:
# Define the image path and load the data
ct_image_path = "../dataset/images/CHUM-002__CT.nii.gz"
pt_image_path = "../dataset/images/CHUM-002__PT.nii.gz"
mask_path = "../dataset/labels/CHUM-002.nii.gz"
ct_obj = nib.load(ct_image_path)
pt_obj = nib.load(pt_image_path)
mask_obj = nib.load(mask_path)
print(f'Type of the image {type(ct_obj)}')

Type of the image <class 'nibabel.nifti1.Nifti1Image'>


In [4]:
# Extract data as numpy ndarray
ct_image_data = ct_obj.get_fdata()
pt_image_data = pt_obj.get_fdata()
mask_data = mask_obj.get_fdata()
resized_pet = cv2.resize(pt_image_data, (ct_image_data.shape[0], ct_image_data.shape[1]))
print(ct_image_data.shape)
print(pt_image_data.shape)
print(resized_pet.shape)
print(mask_data.shape)

(512, 512, 91)
(128, 128, 91)
(512, 512, 91)
(512, 512, 91)


In [5]:
# Get the image shape and print it out
height, width, depth = ct_image_data.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")

The image object has the following dimensions: height: 512, width:512, depth:91


In [6]:
# Define a function to visualize the data
def explore_3dimage(layer):
#     plt.figure(figsize=(20, 10))
#     channel = 3
#     plt.imshow(ct_image_data[:, :, layer], cmap='gray');
#     plt.title('Explore Layers of Brain CT', fontsize=20)
#     plt.axis('off')
    fig, ax = plt.subplots(ncols=2, nrows=2,figsize=(20,20))

    ax[0][0].imshow(apply_window(ct_image_data[:,:,layer]), cmap='gray')
    ax[0][1].imshow(pt_image_data[:,:,layer], cmap='gray')
    ax[1][0].imshow(resized_pet[:,:,layer], cmap='gray')
    ax[1][0].contour(mask_data[:,:,layer])
    ax[1][1].imshow(mask_data[:,:,layer], cmap='gray')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, ct_image_data.shape[2] - 1))

interactive(children=(IntSlider(value=45, description='layer', max=90), Output()), _dom_classes=('widget-inter…

<function __main__.explore_3dimage(layer)>

## Resampled Images

In [7]:
# Define the image path and load the data
ct_image_pathr = "../dataset/images_resampled/CHUM-002__CT.nii.gz"
pt_image_pathr = "../dataset/images_resampled/CHUM-002__PT.nii.gz"
mask_pathr = "../dataset/labels_resampled/CHUM-002.nii.gz"
ct_objr = nib.load(ct_image_pathr)
pt_objr = nib.load(pt_image_pathr)
mask_objr = nib.load(mask_pathr)
print(f'Type of the image {type(ct_objr)}')

Type of the image <class 'nibabel.nifti1.Nifti1Image'>


In [8]:
# Extract data as numpy ndarray
ct_image_datar = ct_objr.get_fdata()
pt_image_datar = pt_objr.get_fdata()
mask_datar = mask_objr.get_fdata()
print(ct_image_datar.shape)
print(pt_image_datar.shape)
print(mask_datar.shape)

(450, 450, 298)
(450, 450, 298)
(450, 450, 298)


In [9]:
# Get the image shape and print it out
height, width, depth = ct_image_datar.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")

The image object has the following dimensions: height: 450, width:450, depth:298


In [10]:
# Define a function to visualize the data
def explore_3dimage(layer):
#     plt.figure(figsize=(20, 10))
#     channel = 3
#     plt.imshow(ct_image_data[:, :, layer], cmap='gray');
#     plt.title('Explore Layers of Brain CT', fontsize=20)
#     plt.axis('off')
    fig, ax = plt.subplots(ncols=3,figsize=(20,20))

    ax[0].imshow(apply_window(ct_image_datar[:,:,layer]), cmap='gray')
    ax[1].imshow(pt_image_datar[:,:,layer], cmap='gray')
    ax[1].contour(mask_datar[:,:,layer])
    ax[2].imshow(mask_datar[:,:,layer], cmap='gray')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, ct_image_datar.shape[2] - 1))

interactive(children=(IntSlider(value=148, description='layer', max=297), Output()), _dom_classes=('widget-int…

<function __main__.explore_3dimage(layer)>

## Evaluate

In [ ]:
import SimpleITK as sitk

In [ ]:
selected_image = sitk.ReadImage(pt_image_pathr)
print('After modification:')
print('\t PET IMAGE:')
print('origin: ' + str(selected_image.GetOrigin()))
print('size: ' + str(selected_image.GetSize()))
print('spacing: ' + str(selected_image.GetSpacing()))
print('direction: ' + str(selected_image.GetDirection()))
print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))
print()
selected_image = sitk.ReadImage(ct_image_pathr)
print('\t CT IMAGE:')
print('origin: ' + str(selected_image.GetOrigin()))
print('size: ' + str(selected_image.GetSize()))
print('spacing: ' + str(selected_image.GetSpacing()))
print('direction: ' + str(selected_image.GetDirection()))
print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))

In [ ]:
selected_image = sitk.ReadImage(pt_image_path)
print('Before modification:')
print('\t PET IMAGE:')
print('origin: ' + str(selected_image.GetOrigin()))
print('size: ' + str(selected_image.GetSize()))
print('spacing: ' + str(selected_image.GetSpacing()))
print('direction: ' + str(selected_image.GetDirection()))
print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))
print()
selected_image = sitk.ReadImage(ct_image_path)
print('\t CT IMAGE:')
print('origin: ' + str(selected_image.GetOrigin()))
print('size: ' + str(selected_image.GetSize()))
print('spacing: ' + str(selected_image.GetSpacing()))
print('direction: ' + str(selected_image.GetDirection()))
print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))